# Cython

* Cython is one of python's *dialects* to bridge between C and python.

* Following code block is an [example](https://stackoverflow.com/questions/35656604/running-cython-in-jupyter-ipython) of the `cython` code.

In [ ]:
%load_ext Cython



In [ ]:
%%cython -a

def geo_prog_cython(double alpha, int n):
    cdef double current = 1.0
    cdef double sum = current
    cdef int i
    for i in range(n):
        current = current * alpha
        sum = sum + current
    return sum



In [ ]:
%%time

geo_prog_cython(0.5, 5)



In [ ]:
def geo_prog_python(alpha, n):
    current = 1.0
    sum = current
    
    for i in range(n):
        current = current * alpha
        sum = sum + current
    return sum



In [ ]:
%%time

geo_prog_python(0.5, 5)


